In [24]:
import requests
import arrow, time, random
from pymongo import MongoClient

client = MongoClient("localhost", 27017)
col = client["tutorial"]["591sample_0825"]

In [17]:
def getParsedData(rawData):
    houseData= [{ 
        "編號": h["id"],
        "標題": h["fulladdress"],
        "租金": int(h["price"].replace(",","")),
        "樓層": h["floor"],
        "坪數": h["area"],
        "地址": h["regionname"] +h["sectionname"]+ h["street_name"] + h["alley_name"] + h["lane_name"] + h["addr_number_name"],
        "連結": "https://rent.591.com.tw/rent-detail-{}.html".format(h["id"])
        }for h in rawData["data"]["data"]]
    return houseData

In [27]:
def printWithTime(msg):
    now = arrow.now().format("YYYY-MM-DD HH:mm:ss")
    print("[{}] {}".format(now,msg))

In [30]:
main_API = "https://rent.591.com.tw/home/search/rsList?is_new_list=1&type=1&kind=0&searchtype=1&region=1&rentprice=3&section=3,10,5&firstRow={}&totalRows=860"
allHouseData=[]
limit = 25
for page in range(10):
    houseData=[]
    API = main_API.format(30*page)
    headers = {
    "Cookie": "urlJumpIp=1; urlJumpIpByTxt=%E5%8F%B0%E5%8C%97%E5%B8%82; is_new_index=1; is_new_index_redirect=1; T591_TOKEN=ugjl0mhvgagjqns6mofrceul64; _ga=GA1.3.1695965811.1564627285; _ga=GA1.4.1695965811.1564627285; _fbp=fb.2.1565668536543.19725422; webp=1; PHPSESSID=3m7c48uum8e93omi5d9c94aro0; user_index_role=1; __auc=c89c609d16cb2e7494dbb461c6b; localTime=2; imgClick=8107221; new_rent_list_kind_test=0; last_search_type=2; _gid=GA1.4.285645481.1566704516; _gid=GA1.3.285645481.1566704516; c10f3143a018a0513ebe1e8d27b5391c=1; DETAIL[1][8122285]=1; DETAIL[1][8015344]=1; DETAIL[1][8096661]=1; DETAIL[1][8114532]=1; DETAIL[1][8112842]=1; DETAIL[1][8099394]=1; DETAIL[1][8091651]=1; DETAIL[1][8121760]=1; DETAIL[1][8105175]=1; DETAIL[1][8011679]=1; user_browse_recent=a%3A5%3A%7Bi%3A0%3Ba%3A2%3A%7Bs%3A4%3A%22type%22%3Bi%3A1%3Bs%3A7%3A%22post_id%22%3Bs%3A7%3A%228011679%22%3B%7Di%3A1%3Ba%3A2%3A%7Bs%3A4%3A%22type%22%3Bi%3A1%3Bs%3A7%3A%22post_id%22%3Bs%3A7%3A%228105175%22%3B%7Di%3A2%3Ba%3A2%3A%7Bs%3A4%3A%22type%22%3Bi%3A1%3Bs%3A7%3A%22post_id%22%3Bs%3A7%3A%228121760%22%3B%7Di%3A3%3Ba%3A2%3A%7Bs%3A4%3A%22type%22%3Bi%3A1%3Bs%3A7%3A%22post_id%22%3Bs%3A7%3A%228091651%22%3B%7Di%3A4%3Ba%3A2%3A%7Bs%3A4%3A%22type%22%3Bi%3A1%3Bs%3A7%3A%22post_id%22%3Bs%3A7%3A%228099394%22%3B%7D%7D; ba_cid=a%3A5%3A%7Bs%3A6%3A%22ba_cid%22%3Bs%3A32%3A%22a06d3b41a6b9600ac62e3c7a9d208d5d%22%3Bs%3A7%3A%22page_ex%22%3Bs%3A48%3A%22https%3A%2F%2Frent.591.com.tw%2Frent-detail-8105175.html%22%3Bs%3A4%3A%22page%22%3Bs%3A48%3A%22https%3A%2F%2Frent.591.com.tw%2Frent-detail-8011679.html%22%3Bs%3A7%3A%22time_ex%22%3Bi%3A1566705295%3Bs%3A4%3A%22time%22%3Bi%3A1566705374%3B%7D; XSRF-TOKEN=eyJpdiI6IldGdDJiQTlnZ2dYeDNUMmUyTGJjSUE9PSIsInZhbHVlIjoiU2tqa21JdGN5b3cwbE1jeDUwNlZEcm5vNHJsRkJzcld2Qm8xY2Q2T1RjZWNQU3Iyd1lIS0tFRXhVYUlVVmw1SHAwdmVoN3BBamk1NzQ3cStaWjZCQ2c9PSIsIm1hYyI6IjRjMjkzYTZmM2ZjMmM0OTM0YTgwOWMwMWI5Y2IxNjFlMWE1MzZlMTQzMTUwNzJhYzA0ZWY4MGNiMjc3YjM5OWUifQ%3D%3D; 591_new_session=eyJpdiI6Ilk5bklvM3J2YnpoWFkreUN1U0V1eFE9PSIsInZhbHVlIjoiTkxybmxlN1ROTUQ5NUVOcnBMNjNBMmR2ZGMzYlJobTlhRFVLcER2NkR5VWhtKzlwUnNONEw2ZmljV0FtSGZrR0xxMXhKVDNnWTRXbHprVmpuOUxLdWc9PSIsIm1hYyI6Ijk0ZjA4NTk3OWYyMTI4Y2Q0ZjcwNTYwNmRhNzFkZDU1MWRmYzgwNzAxN2ZmNmU3Mzk0YjkxN2E1NWQxODFmMjcifQ%3D%3D; _gat_UA-97423186-1=1",
    "Host": "rent.591.com.tw",
    "Referer": "https://rent.591.com.tw/?kind=0&region=1&rentprice=3&section=3,10,5",
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.142 Safari/537.36",
    "X-CSRF-TOKEN": "ovgt2ubUQow4YVFaEskgYRD3ziiWcumJ4309VdlQ",
    "X-Requested-With": "XMLHttpRequest"
}
    time.sleep(random.random()*1.5)
    res2 = requests.get(API, headers = headers)
    data = res2.json()
    houseData = getParsedData(data)
    allHouseData+= houseData
    if len(allHouseData) >= limit:
        col.insert_many(allHouseData)
        allHouseData=[]
        printWithTime("page{} inserted.".format(page+1))
        
if len(allHouseData) >0:
    col.insert_many(allHouseData)
    allHouseData = []
    printWithTime("page{} inserted.".format(page+1))
        
printWithTime("All inserted.")

[2019-08-25 15:08:07] page1 inserted.
[2019-08-25 15:08:10] page2 inserted.
[2019-08-25 15:08:15] page3 inserted.
[2019-08-25 15:08:17] page4 inserted.
[2019-08-25 15:08:19] page5 inserted.
[2019-08-25 15:08:20] page6 inserted.
[2019-08-25 15:08:21] page7 inserted.
[2019-08-25 15:08:23] page8 inserted.
[2019-08-25 15:08:25] page9 inserted.
[2019-08-25 15:08:28] page10 inserted.
[2019-08-25 15:08:28] All inserted.


In [50]:
from bson.objectid import ObjectId
col = client["tutorial"]["591sample_0825"]
#1)高低排序
# list(col.find().sort([("租金",1)]))

# 2)租金大於18000
cursor = col.find({"租金":{"$gt":18000}}, {"租金":1, "連結":1, "坪數":1, "地址":1})
data = list(cursor)
# data

#sorted
sorted(data, key = lambda ele:ele["租金"], reverse = False)


#3) 忠孝東路or 南京東路 上的物件
list(col.find({"地址":{"$regex":".*忠孝東路.*"}}))

### 4) 透過object ID找資料
d = col.find_one({"_id": ObjectId('5d6231ea2ab564bca2f14429')})
print(d)

{'_id': ObjectId('5d6231ea2ab564bca2f14429'), '編號': 8011657, '標題': '忠孝東路三段276巷忠孝東路，SOGO後方，鬧中取靜的好所在', '租金': 14000, '樓層': 5, '坪數': 7, '地址': '台北市大安區忠孝東路三段276巷', '連結': 'https://rent.591.com.tw/rent-detail-8011657.html'}


In [ ]:
from bson.objectid import ObjectId
# 微分析
### 1)取得最低，最高價位的物件 ，從大排到小
# list(    col.find({}).sort([("price",-1)]).limit(2)   )

###2) brouse number >2000
# cursor
cursor = col.find({"browsenum_all":{"$gt": 2000}}, {"link":1,"address_img_title":1,"browsenum_all":1,"price":1,"fulladdress":1})    
# data
data = list(cursor)
# sorted
data
sorted(data, key = lambda ele: ele["price"],reverse = True)
sorted(data, key = lambda ele: ele["browsenum_all"],reverse = True)

### 3) 忠孝東路or 南京東路 上的物件
list(col.find({"fulladdress":{"$regex":".*忠孝東路.*"}}))


### 4) 透過object ID找資料 必須要import from bson.objectid import ObjectId
d = col.find_one({"_id": ObjectId("5d5f6c5ea1c1e5adc5a228ca")})
print(d)

In [57]:
#data sorting
testData1 = [2, -4, 19 ,9,0,-32]
testData2 = ["a","g", "k", "b", "c", "z"]
testData3 = ["Apple", "Book", "Jack", "Flower", "Opera", "apple", "book", "zoo"]

sorted(testData1, key = lambda ele: -ele)
sorted(testData1, key = lambda ele: ele)

print(sorted(testData2))
print(sorted(testData2, reverse = True)) 
print(sorted(testData3))
print(sorted(testData3, reverse = True))

# # ascii as-key-code英文編碼
# print("a", ord("a"))
##########################


['a', 'b', 'c', 'g', 'k', 'z']
['z', 'k', 'g', 'c', 'b', 'a']
['Apple', 'Book', 'Flower', 'Jack', 'Opera', 'apple', 'book', 'zoo']
['zoo', 'book', 'apple', 'Opera', 'Jack', 'Flower', 'Book', 'Apple']


In [61]:
add = lambda a,b: a+b
print(add(3,8))

# testData1
print([n*2 for n in testData1])

# 映射
print(list(map(lambda n : n*2, testData1)))

print(list(map(lambda n: n+"_qq", testData2)))

11
[4, -8, 38, 18, 0, -64]
[4, -8, 38, 18, 0, -64]
['a_qq', 'g_qq', 'k_qq', 'b_qq', 'c_qq', 'z_qq']
